# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-11-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-11-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-12 04:21:00,33.93911,67.709953,57160,2521,51989.0,2650.0,Afghanistan,146.833897,4.410427
1,NaN,NaN,NaN,Albania,2021-04-12 04:21:00,41.15330,20.168300,128393,2317,97723.0,28353.0,Albania,4461.498367,1.804616
2,NaN,NaN,NaN,Algeria,2021-04-12 04:21:00,28.03390,1.659600,118516,3130,82600.0,32786.0,Algeria,270.269512,2.640994
3,NaN,NaN,NaN,Andorra,2021-04-12 04:21:00,42.50630,1.521800,12545,120,11825.0,600.0,Andorra,16236.329515,0.956556
4,NaN,NaN,NaN,Angola,2021-04-12 04:21:00,-11.20270,17.873900,23457,553,22058.0,846.0,Angola,71.371048,2.357505


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56572,56595,56676,56717,56779,56873,56943,57019,57144,57160
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,125842,126183,126531,126795,126936,127192,127509,127795,128155,128393
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,117429,117524,117622,117739,117879,118004,118116,118251,118378,118516


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2495,2496,2497,2508,2512,2512,2516,2521,2521,2521
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2247,2256,2265,2274,2283,2291,2297,2304,2310,2317
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3099,3102,3105,3108,3112,3116,3119,3123,3126,3130


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,51798,51802,51885,51902,51928,51940,51956,51961,51962,51989
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,92500,93173,93842,94431,95035,95600,96129,96672,97206,97723
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,81729,81813,81896,81994,82096,82192,82289,82392,82493,82600


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6606,6617,6619,6620,6644,6675,6702,6710,6715,6723
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20519,20526,20541,20542,20551,20573,20588,20600,20617,20631
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2228,2231,2232,2232,2238,2239,2244,2245,2247,2247


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,99,102,102,102,102,102,103,103,103,103
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,301,301,301,301,301,301,300,300,300,300
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,55,55,55,55,55,55,55,55,55,55


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-12 04:21:00,32.539527,-86.644082,6723,103,NaN,NaN,"Autauga, Alabama, US",12033.506954,1.532054
689,1075.0,Lamar,Alabama,US,2021-04-12 04:21:00,33.779950,-88.096680,1392,34,NaN,NaN,"Lamar, Alabama, US",10083.303151,2.442529
690,1077.0,Lauderdale,Alabama,US,2021-04-12 04:21:00,34.901719,-87.656247,9215,227,NaN,NaN,"Lauderdale, Alabama, US",9937.559987,2.463375


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,31197511,562066,0.0
Brazil,13482023,353137,11878958.0
India,13358805,169275,12081443.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,31197511,562066,0.0,30635445.0,2021-04-12 04:21:00,37.950547,-91.419547
Brazil,13482023,353137,11878958.0,1249928.0,2021-04-12 04:21:00,-12.669522,-48.480493
India,13358805,169275,12081443.0,1108087.0,2021-04-12 04:21:00,23.088275,81.806127
France,5119585,98909,316616.0,4704060.0,2021-04-12 04:21:00,6.430808,-34.730285
Russia,4589209,101282,4216960.0,270967.0,2021-04-12 04:21:00,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3700109,60452,0.0
Texas,2826977,49212,0.0
Florida,2124233,34021,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3700109,60452,0.0,3639657.0,2021-04-12 04:21:00,37.843962,-120.728594
Texas,2826977,49212,0.0,2777765.0,2021-04-12 04:21:00,31.660643,-98.653069
Florida,2124233,34021,0.0,2090212.0,2021-04-12 04:21:00,28.940755,-82.700744
New York,1956594,51117,0.0,1905477.0,2021-04-12 04:21:00,42.544151,-75.474183
Illinois,1279663,23793,0.0,1255870.0,2021-04-12 04:21:00,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1225796,23493,0.0
Arizona,Maricopa,528377,9728,0.0
Illinois,Cook,512773,9755,0.0
Florida,Miami-Dade,460653,5981,0.0
Texas,Harris,382779,5975,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1225796,23493,0.0,1202303.0,2021-04-12 04:21:00,34.308284,-118.228241,6037.0
Arizona,Maricopa,528377,9728,0.0,518649.0,2021-04-12 04:21:00,33.348359,-112.491815,4013.0
Illinois,Cook,512773,9755,0.0,503018.0,2021-04-12 04:21:00,41.841448,-87.816588,17031.0
Florida,Miami-Dade,460653,5981,0.0,454672.0,2021-04-12 04:21:00,25.611236,-80.551706,12086.0
Texas,Harris,382779,5975,0.0,376804.0,2021-04-12 04:21:00,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,57019,127795,118251,12456,23242,1182,2497881,201158,29396,571616,...,4380167,137946,84529,3,172461,2683,264395,5233,89783,37147
2021-04-10,57144,128155,118378,12497,23331,1197,2517300,202167,29405,574755,...,4382880,141380,84724,3,173786,2692,265897,5276,89918,37273
2021-04-11,57160,128393,118516,12545,23457,1198,2532562,202817,29419,577007,...,4384610,144642,84922,3,174887,2693,268132,5357,90029,37288


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,2521,2304,3123,120,549,30,57350,3697,909,9624,...,127284,1363,634,0,1739,35,2812,1022,1226,1535
2021-04-10,2521,2310,3126,120,550,30,57647,3720,909,9662,...,127324,1414,634,0,1759,35,2838,1031,1226,1538
2021-04-11,2521,2317,3130,120,553,30,57779,3735,909,9687,...,127331,1462,634,0,1778,35,2860,1049,1226,1538


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,51961,96672,82392,11732,21589,913,2203926,180160,23080,529191,...,13467,104241,82415,1,155662,2429,229876,1987,85559,34831
2021-04-10,51962,97206,82493,11770,21890,936,2217783,181180,23083,532539,...,13494,104434,82586,1,156472,2429,231288,2027,86813,34862
2021-04-11,51989,97723,82600,11825,22058,938,2233140,181963,23085,535798,...,13494,110277,82760,1,157486,2429,232960,2053,88018,34873


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6606,6617,6619,6620,6644,6675,6702,6710,6715,6723
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9148,9156,9159,9162,9165,9178,9188,9196,9208,9215
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2931,2932,2932,2932,2938,2947,2950,2950,2952,2953
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15444,15452,15457,15459,15468,15468,15480,15491,15510,15524


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,518234,64782,0,848202,331882,3693674,475989,321586,97784,49,...,822085,2822889,389353,20815,2971,632625,374794,145299,643951,56873
2021-04-10,518588,64782,0,849021,332053,3696980,477443,321586,98203,49,...,822085,2825551,389760,20965,2971,634325,376230,145754,644901,56873
2021-04-11,518899,64782,0,849561,332153,3700109,478678,321586,98570,49,...,822085,2826977,390104,21202,2971,635552,376230,146169,645514,56873


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-09        6710   20600    2245  2561   6475    1213   2118   14270   
2021-04-10        6715   20617    2247  2562   6480    1213   2122   14277   
2021-04-11        6723   20631    2247  2562   6483    1213   2122   14284   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-09         3496     1818  ...    2653    611     3085      734   
2021-04-10         3499     1820  ...    2653    611     3085      734   
2021-04-11         3499     1820  ...    2653    611     3085      734   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-09           4085  3660  2142          0      897    640  
2021-04-10           4085  3660  2142          0      897    640  
2021-04-11           4085  3660  2142          0      897    640  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,10686,313,0,17062,5661,60286,6151,7944,1573,0,...,12001,49121,2157,231,26,10451,5322,2737,7375,701
2021-04-10,10712,313,0,17092,5661,60384,6155,7944,1577,0,...,12001,49188,2159,231,26,10458,5322,2742,7379,701
2021-04-11,10712,313,0,17086,5661,60452,6157,7944,1578,0,...,12001,49212,2159,231,26,10472,5322,2745,7380,701


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-09         103     300      55   58    134      40     66     311   
2021-04-10         103     300      55   58    135      40     66     311   
2021-04-11         103     300      55   58    135      40     66     311   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-09          122       43  ...      30     11       31        7   
2021-04-10          122       44  ...      30     11       31        7   
2021-04-11          122       44  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-09             37     9    12          0       26      5  
2021-04-10             37     9    12          0       26      5  
2021-04-11             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.001335,0.002243,0.001143,0.003788,0.005799,0.001695,0.009754,0.005142,0.000204,0.004749,...,-0.001092,0.055787,0.002455,0.0,0.006349,0.005622,0.009076,0.019482,0.002132,0.002564
2021-04-10,0.002192,0.002817,0.001074,0.003292,0.003829,0.012690,0.007774,0.005016,0.000306,0.005491,...,0.000619,0.024894,0.002307,0.0,0.007683,0.003354,0.005681,0.008217,0.001504,0.003392
2021-04-11,0.000280,0.001857,0.001166,0.003841,0.005401,0.000835,0.006063,0.003215,0.000476,0.003918,...,0.000395,0.023073,0.002337,0.0,0.006335,0.000371,0.008406,0.015353,0.001234,0.000402


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.001987,0.003047,0.001282,0.0,0.000000,0.034483,0.003991,0.005986,0.0,0.003964,...,0.000472,0.069020,0.0,NaN,0.011047,0.0,0.011147,0.017928,0.000816,0.001958
2021-04-10,0.000000,0.002604,0.000961,0.0,0.001821,0.000000,0.005179,0.006221,0.0,0.003948,...,0.000314,0.037417,0.0,NaN,0.011501,0.0,0.009246,0.008806,0.000000,0.001954
2021-04-11,0.000000,0.003030,0.001280,0.0,0.005455,0.000000,0.002290,0.004032,0.0,0.002587,...,0.000055,0.033946,0.0,NaN,0.010802,0.0,0.007752,0.017459,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.000096,0.005649,0.001252,0.003421,0.001484,0.000000,0.006826,0.006239,0.000390,0.006675,...,0.002382,0.029632,0.002201,0.0,0.005497,0.0,0.008281,0.021069,0.001322,0.000488
2021-04-10,0.000019,0.005524,0.001226,0.003239,0.013942,0.025192,0.006287,0.005662,0.000130,0.006327,...,0.002005,0.001851,0.002075,0.0,0.005204,0.0,0.006142,0.020131,0.014657,0.000890
2021-04-11,0.000520,0.005319,0.001297,0.004673,0.007675,0.002137,0.006924,0.004322,0.000087,0.006120,...,0.000000,0.055949,0.002107,0.0,0.006480,0.0,0.007229,0.012827,0.013880,0.000316


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,0.000614,0.00389,NaN,0.001537,0.000537,0.000987,0.004084,0.005651,0.003901,0.0,...,0.001364,0.001166,0.001142,0.007064,0.0,0.002443,0.004239,0.003308,0.001858,0.00125
2021-04-10,0.000683,0.00000,NaN,0.000966,0.000515,0.000895,0.003055,0.000000,0.004285,0.0,...,0.000000,0.000943,0.001045,0.007206,0.0,0.002687,0.003831,0.003131,0.001475,0.00000
2021-04-11,0.000600,0.00000,NaN,0.000636,0.000301,0.000846,0.002587,0.000000,0.003737,0.0,...,0.000000,0.000505,0.000883,0.011305,0.0,0.001934,0.000000,0.002847,0.000951,0.00000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-04-09      0.001194  0.000583  0.000446  0.000391  0.000464     0.0   
2021-04-10      0.000745  0.000825  0.000891  0.000390  0.000772     0.0   
2021-04-11      0.001191  0.000679  0.000000  0.000000  0.000463     0.0   

Province_State                                         ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers Cherokee  ...      Park Platte   
2021-04-09     -0.000472  0.000491  0.000572   0.0000  ...  0.001132    0.0   
2021-04-10      0.001889  0.000491  0.000858   0.0011  ...  0.000000    0.0   
2021-04-11      0.000000  0.000490  0.000000   0.0000  ...  0.000000    0.0   

Province_State                                                               \
Admin2         Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-04-09          0.0  0.004104   0.001471  0.001916  0.000935        NaN   
2021-04-10          0.0  0.000000   0.000000  0.000000  0.000000        NaN   
2021-04-11          0.0  0.000000   0.000000  0.000000  0.000000        NaN   

Province_State                      
Admin2          Washakie    Weston  
2021-04-09      0.004479  0.001565  
2021-04-10      0.000000  0.000000  
2021-04-11      0.000000  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,0.001030,0.0,NaN,0.000410,-0.000177,0.002611,0.001302,0.000504,0.003189,NaN,...,0.000333,0.002122,0.003723,0.004348,0.0,0.001437,0.001129,0.000731,0.000678,0.0
2021-04-10,0.002433,0.0,NaN,0.001758,0.000000,0.001626,0.000650,0.000000,0.002543,NaN,...,0.000000,0.001364,0.000927,0.000000,0.0,0.000670,0.000000,0.001827,0.000542,0.0
2021-04-11,0.000000,0.0,NaN,-0.000351,0.000000,0.001126,0.000325,0.000000,0.000634,NaN,...,0.000000,0.000488,0.000000,0.000000,0.0,0.001339,0.000000,0.001094,0.000136,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                \
Admin2         Autauga Baldwin Barbour Bibb    Blount Bullock Butler   
2021-04-09         0.0     0.0     0.0  0.0  0.000000     0.0    0.0   
2021-04-10         0.0     0.0     0.0  0.0  0.007463     0.0    0.0   
2021-04-11         0.0     0.0     0.0  0.0  0.000000     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun Chambers  Cherokee  ...    Park Platte Sheridan   
2021-04-09      0.003226      0.0  0.000000  ...     0.0    0.0      0.0   
2021-04-10      0.000000      0.0  0.023256  ...     0.0    0.0      0.0   
2021-04-11      0.000000      0.0  0.000000  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-09          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-10          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-11          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.001306,0.002218,0.001074,0.003681,0.00533,0.002152,0.009288,0.005221,0.000218,0.004856,...,-0.000211,0.042225,0.002330,2.328306e-10,0.006877,0.004575,0.009873,0.018226,0.002200,0.001876
2021-04-10,0.001749,0.002517,0.001074,0.003486,0.00458,0.007421,0.008531,0.005118,0.000262,0.005174,...,0.000204,0.033559,0.002318,1.164153e-10,0.007280,0.003965,0.007777,0.013221,0.001852,0.002634
2021-04-11,0.001015,0.002187,0.001120,0.003664,0.00499,0.004128,0.007297,0.004167,0.000369,0.004546,...,0.000299,0.028316,0.002328,5.820766e-11,0.006808,0.002168,0.008091,0.014287,0.001543,0.001518


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.001656,0.003105,0.001184,0.004340,0.001527,0.018357,0.004091,0.006184,5.429935e-35,0.003830,...,0.000406,0.052275,0.000723,NaN,0.009762,4.720517e-68,0.009744,0.016960,0.000853,0.001403
2021-04-10,0.000828,0.002855,0.001072,0.002170,0.001674,0.009179,0.004635,0.006203,2.714967e-35,0.003889,...,0.000360,0.044846,0.000362,NaN,0.010631,2.360259e-68,0.009495,0.012883,0.000426,0.001679
2021-04-11,0.000414,0.002942,0.001176,0.001085,0.003564,0.004589,0.003462,0.005117,1.357484e-35,0.003238,...,0.000208,0.039396,0.000181,NaN,0.010716,1.180129e-68,0.008624,0.015171,0.000213,0.000839


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-09,0.000244,0.005769,0.001215,0.004340,0.002255,0.004652,0.006309,0.005237,0.000427,0.006360,...,0.002878,0.027626,0.002117,0.0,0.005636,0.001001,0.007895,0.025619,0.001069,0.000469
2021-04-10,0.000131,0.005646,0.001220,0.003790,0.008099,0.014922,0.006298,0.005449,0.000278,0.006344,...,0.002442,0.014739,0.002096,0.0,0.005420,0.000501,0.007019,0.022875,0.007863,0.000680
2021-04-11,0.000325,0.005482,0.001259,0.004231,0.007887,0.008529,0.006611,0.004885,0.000182,0.006232,...,0.001221,0.035344,0.002101,0.0,0.005950,0.000250,0.007124,0.017851,0.010872,0.000498


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,0.000661,0.003618,NaN,0.001164,0.000558,0.000849,0.003869,0.004546,0.003519,2.699938e-118,...,0.001570,0.001322,0.001173,0.007580,0.002617,0.002561,0.003630,0.003010,0.001710,0.001408
2021-04-10,0.000672,0.001809,NaN,0.001065,0.000537,0.000872,0.003462,0.002273,0.003902,1.349969e-118,...,0.000785,0.001132,0.001109,0.007393,0.001308,0.002624,0.003731,0.003071,0.001593,0.000704
2021-04-11,0.000636,0.000905,NaN,0.000850,0.000419,0.000859,0.003024,0.001136,0.003820,6.749845e-119,...,0.000392,0.000819,0.000996,0.009349,0.000654,0.002279,0.001865,0.002959,0.001272,0.000352


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-09      0.002451  0.000653  0.001025  0.000968  0.000521  0.000641   
2021-04-10      0.001598  0.000739  0.000958  0.000679  0.000647  0.000321   
2021-04-11      0.001395  0.000709  0.000479  0.000340  0.000555  0.000160   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-09      0.000500  0.000502  0.000429  0.000494  ...  0.001075   
2021-04-10      0.001194  0.000496  0.000643  0.000797  ...  0.000537   
2021-04-11      0.000597  0.000493  0.000322  0.000398  ...  0.000269   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-09      0.000008  0.000138  0.005503   0.001936  0.001903  0.000733   
2021-04-10      0.000004  0.000069  0.002751   0.000968  0.000952  0.000367   
2021-04-11      0.000002  0.000035  0.001376   0.000484  0.000476  0.000183   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-09           -1.0  0.002555  0.001624  
2021-04-10           -1.0  0.001278  0.000812  
2021-04-11           -1.0  0.000639  0.000406  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-09,0.001185,7.488328e-08,NaN,0.000902,0.000255,0.002274,0.001016,0.000624,0.001823,NaN,...,0.000940,0.002001,0.003300,0.002781,1.220703e-06,0.001812,0.001428,0.002572,0.001022,0.000096
2021-04-10,0.001809,3.744164e-08,NaN,0.001330,0.000128,0.001950,0.000833,0.000312,0.002183,NaN,...,0.000470,0.001682,0.002114,0.001390,6.103516e-07,0.001241,0.000714,0.002199,0.000782,0.000048
2021-04-11,0.000904,1.872082e-08,NaN,0.000490,0.000064,0.001538,0.000579,0.000156,0.001408,NaN,...,0.000235,0.001085,0.001057,0.000695,3.051758e-07,0.001290,0.000357,0.001647,0.000459,0.000024


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2021-04-09      0.002688 -0.000830  0.000018 -4.838394e-13  0.002857   
2021-04-10      0.001344 -0.000415  0.000009 -2.419197e-13  0.005160   
2021-04-11      0.000672 -0.000208  0.000005 -1.209598e-13  0.002580   

Province_State                                                           ...  \
Admin2           Bullock        Butler  Calhoun  Chambers      Cherokee  ...   
2021-04-09      0.000025  3.720158e-15  0.00168  0.002728  3.633221e-07  ...   
2021-04-10      0.000013  1.860079e-15  0.00084  0.001364  1.162809e-02  ...   
2021-04-11      0.000006  9.300395e-16  0.00042  0.000682  5.814044e-03  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-04-09      1.315586e-07  5.339914e-24  9.936129e-10  4.365031e-30   
2021-04-10      6.577930e-08  2.669957e-24  4.968064e-10  2.182515e-30   
2021-04-11      3.288965e-08  1.334978e-24  2.484032e-10  1.091258e-30   

Province_State                                                    \
Admin2         Sweetwater         Teton         Uinta Unassigned   
2021-04-09       0.000027  1.393400e-17  7.081173e-25  -0.501961   
2021-04-10       0.000014  6.966999e-18  3.540586e-25  -0.501961   
2021-04-11       0.000007  3.483500e-18  1.770293e-25  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-09      2.712067e-22  5.759824e-21  
2021-04-10      1.356033e-22  2.879912e-21  
2021-04-11      6.780167e-23  1.439956e-21  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
